In [32]:
import pandas as pd
import numpy as np
import movie_stars
import warnings
from astropy.io import fits
warnings.filterwarnings("ignore")

time_to_integrate = np.linspace(0, -60, 200)


# apogee catalog to crossmatch with
df_apogee = pd.read_csv('/Users/cam/Downloads/apogee_dr17.csv')

# Open Clusters

In [33]:
df_open_clusters = pd.read_csv('/Users/cam/Downloads/cg_ocs_full.csv')


df_open_clusters['age_myr'] = df_open_clusters['age_myr']/1e6
df_open_clusters = df_open_clusters.loc[(df_open_clusters['n_rv'] >= 5)]
df_open_clusters = df_open_clusters.loc[(np.abs(df_open_clusters.rv_chosen_med) < 200) & (df_open_clusters.rv_chosen_med.notnull()) & (df_open_clusters['parallax_med'] > 0.)]
df_oc_young = df_open_clusters.loc[df_open_clusters.age_myr <= 80]


open_clusters_key_dict = {'ra_med': 'ra',
                          'dec_med': 'dec',
                          'parallax_med': 'parallax',
                          'pmra_med': 'pmra',
                          'pmdec_med': 'pmdec',
                          'rv_chosen_med': 'radial_velocity',
                          'age_myr': 'age',
                          'cluster': 'group_name'}
oc_marker_props = {'size':np.log10(df_oc_young.n.values),
                   'color': '#00CED1',
                   'opacity': 1.,
                   'symbol': 'age-based',
                   'line' : dict(width=0.)
                   }


open_clusters_young = movie_stars.MovieStar(name='CG Open Clusters (Age [Myr] < 80)',
                                            df=df_oc_young,
                                            key_dict=open_clusters_key_dict,
                                            time_to_integrate=time_to_integrate,
                                            marker_properties=oc_marker_props
                                            )

In [44]:
# df_oc_young['n'].values

array([ 18.,   8.,  19.,  54.,  44.,  19.,   8.,  15.,   8.,   5.,  19.,
         8.,  16.,  29.,  16.,   6.,  77.,   5.,   5.,   5.,  49.,  13.,
        31., 195.,   6.,   6.,  67.,  16.,   7.,   6.,  54.,   8.,  69.,
       149.,  51., 257., 561.,  46.,  23., 129.,   7.,  20.,  89., 134.,
        66.,  68.,  87.,  16.,   7.,  53.,  11.,   9.,   9.,  33.,  23.,
       112.,  22.,  18.,  15.,  16.,  12.,  59.,   8.,  15.,   6.,   5.,
        75.,  41.,  29.,  24.,   5.,  11.,   5.,  21.,  14.,   6.,  16.,
        14.,  16.,   7.,  16.,   6.,   9.,  22.,   5.,   8.,  53.,   7.])

In [34]:
# cluster_list_1 = ['Alessi_21', 'ASCC_58', 'Alessi_5', 'BH_99', 'UPK_82', 'ASCC_105', 'Roslund_5', 'Alessi_24', 'UBC_1', 'UPK_545', 'Melotte_22', 'UPK_612']
# df_oc_1 = df_oc_young.loc[df_oc_young.cluster.isin(cluster_list_1)]

# oc_mp_1 = oc_marker_props.copy()
# oc_mp_1['color'] = 'red'
# open_clusters_1 = movie_stars.MovieStar(name='Story 1',
#                                             df=df_oc_1,
#                                             key_dict=open_clusters_key_dict,
#                                             time_to_integrate=time_to_integrate,
#                                             marker_properties=oc_mp_1
#                                             )
#df_oc_guo = df_oc_young

# Sebastian Sco-Cen

In [35]:
df_sc = pd.read_csv('/Users/cam/Downloads/sc_sebastian_apogee_and_sos.csv')


# creating movie class
sco_cen_key_dict = {'ra' : 'ra',
                    'dec' : 'dec',
                    'parallax' : 'parallax',
                    'pmra' : 'pmra',
                    'pmdec' : 'pmdec',
                    'rv_chosen' : 'radial_velocity',
                    'label' : 'group_name',
                    'age' : 'age'
}
sco_cen_marker_props = {'size' : 9, 
                        'opacity' : 1., 
                        'symbol' : 'age-based', 
                        'color' : 'magenta',
                        'line' : dict(width=0.)
                        }

sco_cen = movie_stars.MovieStar(name = 'Sebastian Sco-Cen', 
                                time_to_integrate = time_to_integrate, 
                                df = df_sc, 
                                key_dict = sco_cen_key_dict, 
                                marker_properties = sco_cen_marker_props
                                )


sco_cen.marker_properties['symbol'] == 'age-based'


True

# Kerr+2021 Groups (besides Sco-Cen)

In [36]:
df_kerr = pd.read_csv('/Users/cam/Desktop/gaia_3d_plot/data/kerr_sos_edr3.csv')
df_kerr_apogee = pd.read_csv('/Users/cam/Downloads/sco_cen_kerr_apogee.csv')
#(df_kerr.TLC != 22) 
df_kerr = df_kerr.loc[(df_kerr.EOM > 0)]
df_kerr = pd.merge(left = df_kerr, right = df_apogee, how = 'left', left_on = 'source_id', right_on = 'gaiaedr3_source_id')
df_kerr = df_kerr.loc[(df_kerr.parallax > 0.)]

df_kerr_apogee = df_kerr.loc[df_kerr['vhelio_avg'].notnull()]
df_kerr_apogee['rv'] = df_kerr_apogee['vhelio_avg']
df_kerr_apogee['rv_err'] = df_kerr_apogee['verr']

df_kerr_sos = df_kerr.loc[(df_kerr['RVcor_merged'].notnull()) & (df_kerr['vhelio_avg'].isnull())]
df_kerr_sos['rv'] = df_kerr_sos['RVcor_merged']
df_kerr_sos['rv_err'] = df_kerr_sos['ERRVcor_merged']
df_kerr = pd.concat([df_kerr_apogee, df_kerr_sos])

df_kerr = df_kerr.groupby(['EOM']).median().reset_index()

kerr_key_dict = {'ra' : 'ra',
                    'dec' : 'dec',
                    'parallax' : 'parallax',
                    'pmra' : 'pmra',
                    'pmdec' : 'pmdec',
                    'rv' : 'radial_velocity',
                    'EOM' : 'group_name',
                    'Age' : 'age'
}
kerr_marker_props = {'size' : 9, 
                        'opacity' : 1., 
                        'symbol' : 'age-based', 
                        'color' : '#4169E1',
                        'line' : dict(width = 0.)
                        }

kerr = movie_stars.MovieStar(name = 'Kerr+2021', 
                                time_to_integrate = time_to_integrate, 
                                df = df_kerr, 
                                key_dict = kerr_key_dict, 
                                marker_properties = kerr_marker_props)



# Nuria Groups

In [37]:
df_nuria = pd.read_csv('/Users/cam/Downloads/table_YLAs_equ_family2.csv')
df_nuria['age'] = np.ones(len(df_nuria))*35

nuria_key_dict = {'ra': 'ra',
    'dec': 'dec',
    'parallax': 'parallax',
    'pmra': 'pmra',
    'pmdec': 'pmdec',
    'RV': 'radial_velocity',
    'age': 'age',
    'Group': 'group_name'}
nuria_marker_props = {'size': 7,
                   'color': 'orange',
                   'opacity': 1.,
                   'symbol': 'age-based',
                   'line' : dict(width=0.)
                   }

nuria = movie_stars.MovieStar(name = 'Nuria Groups', time_to_integrate=time_to_integrate, df = df_nuria, key_dict=nuria_key_dict, marker_properties=nuria_marker_props)



# Swiggum/Chen Orion

In [38]:
df_orion = pd.read_csv('/Users/cam/Desktop/astro_research/orion/orion_full_groups_with_age.csv')
df_orion = df_orion.loc[df_orion.apogee2_radial_velocity.notnull()]
df_orion = df_orion.groupby(['label']).median().reset_index()
df_orion = df_orion.loc[df_orion.probability > .2]

orion_key_dict = {'ra': 'ra',
    'dec': 'dec',
    'parallax': 'parallax',
    'pmra': 'pmra',
    'pmdec': 'pmdec',
    'apogee2_radial_velocity': 'radial_velocity',
    'age': 'age',
    'label': 'group_name'}
orion_marker_props = {'size': 9,
                   'color': '#8A2BE2',
                   'opacity': 1.,
                   'symbol': 'age-based',
                    'line' : dict(width=0.)
                   }
orion = movie_stars.MovieStar(name = 'Swiggum/Chen Orion', time_to_integrate=time_to_integrate, df = df_orion, key_dict=orion_key_dict, marker_properties=orion_marker_props)

In [39]:
# Hao clusters
hdu_oc_hao = fits.open('/Users/cam/Downloads/asu-33.fit')
df_oc_hao = pd.DataFrame(hdu_oc_hao[1].data)

df_oc_hao['age'] = 10**df_oc_hao.age.values/1e6
df_oc_hao = df_oc_hao.loc[(df_oc_hao.age < 80) & (df_oc_hao.N > 5)]
df_oc_hao = df_oc_hao.loc[df_oc_hao.RV != 0.]

hao_key_dict = {'RA_ICRS': 'ra',
    'DE_ICRS': 'dec',
    'plx': 'parallax',
    'pmRA': 'pmra',
    'pmDE': 'pmdec',
    'RV': 'radial_velocity',
    'age': 'age',
    'Cluster': 'group_name'}
hao_marker_props = {'size': 7,
                   'color': '#00CED1',
                   'opacity': 1.,
                   'symbol': 'age-based',
                   'line' : dict(width=0.)
                   }
hao = movie_stars.MovieStar(name = 'Hao OCs (< 80 Myr)', time_to_integrate=time_to_integrate, df = df_oc_hao, key_dict=hao_key_dict, marker_properties=hao_marker_props)

In [40]:
# He Clusters
df_oc_he_full = pd.read_csv('/Users/cam/Downloads/he_oc_rv_full.csv')
df_oc_he = df_oc_he_full.groupby('Cluster').median().reset_index()
df_oc_he = df_oc_he.loc[df_oc_he['rv_chosen'].notnull()]
df_oc_he = df_oc_he.loc[(df_oc_he.age < 80) & (df_oc_he.n_rv > 4)]
df_oc_he = df_oc_he.drop(columns=['ra'])
he_key_dict = {'_RA_icrs': 'ra',
    '_DE_icrs': 'dec',
    'Plx': 'parallax',
    'pmRA': 'pmra',
    'pmDE': 'pmdec',
    'rv_chosen': 'radial_velocity',
    'age': 'age',
    'Cluster': 'group_name'}
he_marker_props = {'size': 7,
                   'color': '#00CED1',
                   'opacity': 1.,
                   'symbol': 'age-based',
                   'line' : dict(width=0.)
                   }
he = movie_stars.MovieStar(name = 'He OCs', time_to_integrate=time_to_integrate, df = df_oc_he, key_dict=he_key_dict, marker_properties=he_marker_props)



In [41]:
# make movie
movie_save_path = '/Users/cam/Desktop/astro_research/radcliffe/movies/galactic_motions_movie.html'
data_list = [open_clusters_young, kerr, nuria, orion, sco_cen]

annotations = [dict(x=-272, y=120, z=50, text = 'Radcliffe Wave (t = 0)', xanchor = 'center', opacity = .8, font = dict(size=10, color = 'red'), showarrow = False),
               dict(x = 235, y = 77, z = 44, text = 'The Split (t = 0)', xanchor = 'center', opacity = .8, font = dict(size=10), showarrow = False),
               dict(x = 2144, y = -5768, z = 60, text = 'Compact Group? (t = -26.5)', xanchor = 'center', opacity = 1., font = dict(size=8), showarrow = True),
               dict(x = 6185, y = -7507, z = 20, text = 'Similar gas-origins for Sco-Cen and OCs? (t = -48.5)', xanchor = 'center', opacity = 1., font = dict(size=8), showarrow = True),
               dict(x = 6929, y = -7574, z = 0, text = 'Similar gas-origins for Sco-Cen and OCs? (t = -48.5)', xanchor = 'center', opacity = 1., font = dict(size=8), showarrow = True),
               dict(x = 379, y = -3096, z = -94, text = 'Sun passes through Orion (t=-13.3)', xanchor = 'center', opacity = 1., font = dict(size=8), showarrow = True)
               ]

movie = movie_stars.Movie(
    movie_stars=data_list, time=time_to_integrate, movie_save_path=movie_save_path,
    center_star='Kerr+2021', annotations=None, xyz_ranges=[3000,3000,500]
    )
movie.make_movie()

ValueError: 
    Invalid value of type 'numpy.ndarray' received for the 'size' property of scatter3d.marker
        Received value: array([array([1.25527251, 0.90308999, 1.2787536 , 1.73239376, 1.64345268,
              1.2787536 , 0.90308999, 1.17609126, 0.90308999, 0.69897   ,
              1.2787536 , 0.90308999, 1.20411998, 1.462398  , 1.20411998,
              0.77815125, 1.88649073, 0.69897   , 0.69897   , 0.69897   ,
              1.69019608, 1.11394335, 1.49136169, 2.29003461, 0.77815125,
              0.77815125, 1.8260748 , 1.20411998, 0.84509804, 0.77815125,
              1.73239376, 0.90308999, 1.83884909, 2.17318627, 1.70757018,
              2.40993312, 2.74896286, 1.66275783, 1.36172784, 2.11058971,
              0.84509804, 1.30103   , 1.94939001, 2.1271048 , 1.81954394,
              1.83250891, 1.93951925, 1.20411998, 0.84509804, 1.72427587,
              1.04139269, 0.95424251, 0.95424251, 1.51851394, 1.36172784,
              2.04921802, 1.34242268, 1.25527251, 1.17